In [113]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [114]:
import statsmodels.api as sm
import sklearn.model_selection as skm
import sklearn.linear_model as skl
from sklearn.preprocessing import StandardScaler
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from functools import partial
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from ISLP.models import \
    (Stepwise,
     sklearn_selected,
     sklearn_selection_path)
from l0bnb import fit_path
from sklearn.model_selection import train_test_split

In [ ]:
### From the ISLP book ###
def CE(estimator, X, Y): #Cp is negative, since sklearn tries to maximize the score, but we want to minimize Cp
    "Cross Entropy"
    Yhat = estimator.predict(X)
    # print(Yhat[Yhat<0])
    maskIsMigrator = Y == 1
    maskIsNotMigrator = Y == 0
    CrossEntropy = np.mean(
        Y[maskIsMigrator]*np.log(Yhat[maskIsMigrator])
        ) + np.mean(
            (1-Y[maskIsNotMigrator])*np.log(Yhat[maskIsNotMigrator])
            )
    print(CrossEntropy)
    return CrossEntropy

    # maskIsMigrator = y_test == 1
    # maskIsNotMigrator = y_test == 0
    # predictions = regr.predict(sm.add_constant(DataFinalTest[list(feature_set)]))
    
    # CrossEntropy = -np.mean(
    #     y_test[maskIsMigrator]*np.log(predictions[maskIsMigrator])
    #     ) - np.mean(
    #         (1-y_test[maskIsNotMigrator])*np.log(1-predictions[maskIsNotMigrator])
    #         )
    

In [116]:
Df = pd.read_csv('Faellesdata_cleaned.csv')

In [117]:
x_train, x_test, y_train, y_test = train_test_split(Df.drop('IsMigratorInt', axis=1), Df['IsMigratorInt'], test_size=0.2, random_state=42)

In [118]:
### From the ISLP book ###
design = MS(x_train).fit(x_train, y_train)
Y = np.array(y_train)
X = design.transform(x_train, y_train)
# sigma2 = sm.GLM(Y, X, family = sm.families.Binomial()).fit().scale
Cross_Entropy = partial(CE)

strategy = Stepwise.first_peak(design,
                               direction = 'forward',
                               max_terms = len(design.terms))

In [119]:
#Forward selection using MSE
fit = partial(sm.GLM, family = sm.families.Binomial())
Y_MSE = sklearn_selected(fit, strategy)
Y_MSE.fit(X, Y)
Y_MSE.selected_state_

('Rcyl',
 'age',
 'cfe',
 'ch',
 'feh',
 'mass',
 'mgfe',
 'mgh',
 'ofe',
 'oh',
 'phi',
 'vRcyl',
 'vphi',
 'vz',
 'z')

In [120]:
# np.log(-1)

In [141]:
#Forward selection using Cross Entropy'
fit = partial(sm.GLM, family = sm.families.Binomial())
Y_CE = sklearn_selected(fit,
                        strategy,
                        scoring = CE)
Y_CE.fit(X, Y)
Y_CE.selected_state_

-3.4545044049711917
-3.45480406081934
-3.455055913534667
-3.5909143329488558
-3.4556613616210123
-3.4546436016186552
-3.512909264797308
-3.472051800799199
-3.4570109053209457
-3.4545428365984474
-3.454532281026843
-3.4546008139553117
-3.4545044060569188
-3.4871985943393797
-3.4553610215498614
-3.4566424560744586
-3.4545044049711917
-3.45480406081934
-3.455055913534667
-3.5909143329488558
-3.4556613616210123
-3.4546436016186552
-3.512909264797308
-3.472051800799199
-3.4570109053209457
-3.4545428365984474
-3.454532281026843
-3.4546008139553117
-3.4545044060569188
-3.4871985943393797
-3.4553610215498614
-3.4566424560744586


()

In [122]:
Y_CE.selected_state_

()

In [123]:
#We repeat the same, but in the backwards direction
strategy = Stepwise.first_peak(design,
                                 direction = 'backwards',
                                 max_terms = len(design.terms))

In [124]:
Y_MSE = sklearn_selected(sm.GLM, strategy)
Y_MSE.fit(X, Y)
Y_MSE.selected_state_

('Rcyl',
 'age',
 'cfe',
 'ch',
 'feh',
 'mass',
 'mgfe',
 'mgh',
 'ofe',
 'oh',
 'phi',
 'vRcyl',
 'vphi',
 'vz',
 'z')

In [125]:
#Backwards selection using Cp
Y_Cp = sklearn_selected(sm.GLM,
                        strategy,
                        scoring = neg_Cp)
Y_Cp.fit(X, Y)
Y_Cp.selected_state_

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\arraylike.py:399: Ru

()